<a href="https://colab.research.google.com/github/llaryanll/lunar-lander-v2/blob/code/Deep_Q_Learning_for_Lunar_Landing_Partial_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


### Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

In [3]:
class Network(nn.Module):
    def __init__(self, state_size, action_size, seed=42): #defining the architecture of the neural network
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, state): #forward propagation in the neural network
        x = self.fc1(state)
        x = F.relu(x) #rectifier activation function -> first layet
        x = self.fc2(x)
        x = F.relu(x)   #second layer activation now begins
        return self.fc3(x);

### Creating the architecture of the Neural Network

## Part 2 - Training the AI

### Setting up the environment

In [4]:
import gymnasium as gym
env = gym.make("LunarLander-v2")
state_shape = env.observation_space.shape
state_size  =  env.observation_space.shape[0];
number_actions = env.action_space.n
print("State shape ",state_shape)
print("State size ",state_size)
print("Number of actions ",number_actions)

State shape  (8,)
State size  8
Number of actions  4


### Initializing the hyperparameters

In [5]:
learning_rate = 5e-4 #based on experimentation
minibatch_size = 100 #conventional
discount_factor = 0.99
replay_buffer_size = int(1e5) #these many experiences will be there
interpolation_parameter = 1e-3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Implementing Experience Replay

In [6]:
class ReplayMemory(object):

  def __init__(self,capacity):
    self.device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity=capacity
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0] #removing the oldest experience when capacity has peaked


  def sample(self, batch_size):
    experiences = random.sample(self.memory, k=batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones



### Implementing the DQN class

In [7]:
class Agent(): #this class defines the behaviour of the entire agent (Rocket Model in this case)

  def __init__(self, state_size, action_size):
      self.device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      self.state_size =  state_size
      self.action_size= action_size

      #q-learning starts

      self.local_qnetwork = Network(state_size, action_size).to(self.device) #this network will select the actions
      self.target_qnetwork = Network(state_size, action_size).to(self.device) #this will calculate values that will be used in training the local model
      self.optimizer =  optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate) #the double network stabilizes the entire project

      self.memory =  ReplayMemory(replay_buffer_size)
      self.t_step=0

  def step(self, state, action, reward, next_state, done): #stores all experiences and learns from them
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step+1) % 4 #learning after all 4 steps
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences  = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon=0.):  #this will help the agent choose the next action to take depending upon the current state of the network
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device) #extra dimension for batch
    self.local_qnetwork.eval() #evaluation mode
    with torch.no_grad():
      action_values = self.local_qnetwork(state) #forward propagation
    self.local_qnetwork.train()
    if random.random() > epsilon: #epsilon greedy policy is chosen for selecting the best Q-Value
      return np.argmax(action_values.cpu().data.numpy()) #exploitation
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self,experiences, discount_factor):
    states, next_states, actions, rewards, dones =  experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (discount_factor * next_q_targets * (1-dones))
    q_expected =  self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected,q_targets)
    self.optimizer.zero_grad()

    #back propagation (super easy for some reason)
    loss.backward()
    self.optimizer.step() #update model params
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter) #update params

  def soft_update(self, local_model, target_model, interploation_paramater): #this will update the target q network params by blending them with those of local q network which would prevent abrupt changes that mightde-stabilise the entire model
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0-interpolation_parameter) * target_param.data)







### Initializing the DQN agent

In [8]:
agent = Agent(state_size, number_actions) #initialise

### Training the DQN agent

In [9]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100) #basic dequq

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon) #do something
    next_state, reward, done ,_, _ = env.step(action)
    agent.step(state, action, reward, next_state, done) #training
    state =  next_state
    score += reward
    if done:
      break

  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value*epsilon) #ensuring that epsilon doesnt reach its limit

  print("\rEpisode {}\tAverage Score: {:.2f}".format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
      print("\rEpisode {}\tAverage Score: {:.2f}".format(episode, np.mean(scores_on_100_episodes)))

  if np.mean(scores_on_100_episodes) >= 200.0 :
    print("\nEnvironmnet solved in {:d} episodes!\tAverage Score: {:.2f}".format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Episode 100	Average Score: -155.91
Episode 200	Average Score: -77.11
Episode 300	Average Score: -37.89
Episode 400	Average Score: 60.69
Episode 500	Average Score: 148.28
Episode 600	Average Score: 182.84
Episode 664	Average Score: 200.00
Environmnet solved in 564 episodes!	Average Score: 200.00


## Part 3 - Visualizing the results

In [10]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()